In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [6]:
%run ../load_anno_mi.py

In [7]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-02 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-02 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-02 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-02 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-02 00:00:34


In [8]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: Fine-tune BERT for classification

In [9]:
num_epochs = 10
seeds = [0, 1, 12, 123, 1234]
validation_metric = "f1"

In [10]:
bert_classifier = fine_tune_transformer_average_seed(num_epochs=num_epochs,
                                                     pretrained_model_name="bert-base-uncased",
                                                     df=anno_mi,
                                                     feature_name="utterance_text",
                                                     label_column="client_talk_type",
                                                     seeds=seeds,
                                                     path_indices=client_index,
                                                     k_fold=True,
                                                     validation_metric=validation_metric,
                                                     results_output=f"{output_dir}/bert_classifier.csv",
                                                     device=device,
                                                     verbose=False)

[INFO] Setting return_special_tokens_mask=True
[INFO] Tokenizing the dataset...


Map:   0%|          | 0/6725 [00:00<?, ? examples/s]

[INFO] Saving the tokenized text for each sentence into `.df['tokens']`...


Map:   0%|          | 0/6725 [00:00<?, ? examples/s]

[INFO] Creating tokenized dataframe and setting in `.tokenized_df` attribute...
[INFO] Note: 'text_id' is the column name for denoting the corresponding text id
[INFO] Setting up TrainingArguments object and saving to `.training_args`.
[INFO] Setting up Trainer object, and saving to `.trainer`.
[INFO] Training model with 109484547 parameters...


...

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.737274,0.695383,0.572140
2,No log,0.723230,0.699324,0.590745
3,No log,0.774364,0.701577,0.594028
4,No log,0.852137,0.705518,0.590349
5,No log,0.889373,0.693694,0.590141
6,No log,0.909303,0.688063,0.570859
7,No log,0.959890,0.690315,0.586734
8,No log,0.971887,0.692005,0.583439
9,0.407900,0.993152,0.689752,0.591994
10,0.407900,1.006553,0.695946,0.592283


[INFO] Training completed!


  0%|          | 0/1345 [00:00<?, ?it/s]

saving the results dataframe to CSV in client_talk_type_output/bert_classifier.csv


In [11]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,k_fold
0,0.701413,0.576948,"[0.8060861120103594, 0.48940998487140697, 0.43...",0.627257,"[0.7396039603960396, 0.6629098360655737, 0.479...",0.557467,"[0.8857007351197533, 0.38788968824940045, 0.39...",0,True
0,0.700372,0.555906,"[0.8034334763948497, 0.5197934595524957, 0.344...",0.630695,"[0.7336860670194003, 0.6103476151980598, 0.548...",0.530554,"[0.8878349537585961, 0.4526378896882494, 0.251...",1,True
0,0.704535,0.582494,"[0.8051948051948052, 0.5374241402562374, 0.404...",0.624158,"[0.7512836311357568, 0.6140215716486903, 0.507...",0.560721,"[0.8674413089874318, 0.47781774580335734, 0.33...",12,True
0,0.710483,0.590863,"[0.8100498158977693, 0.5188437614343214, 0.443...",0.640952,"[0.7454654175802272, 0.6657276995305165, 0.511...",0.567871,"[0.8868864121413327, 0.4250599520383693, 0.391...",123,True
0,0.696952,0.604289,"[0.7971882922332334, 0.5374659400544959, 0.478...",0.612796,"[0.7753866845998655, 0.6222397476340694, 0.440...",0.605297,"[0.8202513635285749, 0.47302158273381295, 0.52...",1234,True


In [12]:
bert_classifier["f1"].mean()

0.5821000015826902

In [13]:
bert_classifier["precision"].mean()

0.6271717229743248

In [14]:
bert_classifier["recall"].mean()

0.5643821405492904

In [15]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.8043905 , 0.52058746, 0.42132205])

In [16]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.74908515, 0.63504929, 0.49738072])

In [17]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.86962295, 0.44328537, 0.3802381 ])